# Open Weather Map API

## 1. create a dummy 'cities' DataFrame

In [ ]:
import pandas as pd
!pip install PyMySQL
import pymysql
import sqlalchemy 

In [ ]:
# sample cities DataFrame for exercises

cities = pd.DataFrame(data = {'City_id' : [1],'City' : ['Berlin'], 'Country_code' : ['DE']})

,City_id,City,Country_code
0,1,Berlin,DE


In [ ]:
schema="gans_database"   # name of the database you want to use here
host="127.0.0.1"        # to connect to your local server
user="root"
password="SharpGajanan9860" # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

## 2. API key

Don't forget that for OpenWeatherAPI you need an API key

In [ ]:
# please use your own API key
OWM_key = 'cf36567c4cb288fd1cd39a18cbfc1b5b'

## 3. Different approaches

As we saw in the Spotify example, for some API's there could be a Python wrapper that can help you get the data in easier and simpler way.  
Here, we will see two different approaches to OpenWeather API:
1. with PyOWM - Python wrapper library
2. without the wrapper

### 3.1 Approach with a wrapper (PyOWM)

Documentation for PyOWM you can find here: https://pyowm.readthedocs.io/en/latest/

In [ ]:
!pip install -U pyowm

from pyowm.owm import OWM

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.5 MB 27.7 MB/s 


If you go to https://pyowm.readthedocs.io/en/latest/v3/code-recipes.html#weather_data you can get code example to get example how to get wind speed in 3h from now:

In [ ]:
owm = OWM(OWM_key)
mgr = owm.weather_manager()
one_call = mgr.one_call(lat=52.5244, lon=13.4105)

one_call.forecast_hourly[3].wind().get('speed', 0)

2.59

Problem with PyOWM is that except for few code examples, they don't provide much details about their classes or methods, so working with this wrapper would have to include a lot of testing and exploration on your own, or even looking into raw code on their GitHub.

#### *BONUS* If you still want to use a wrapper, check the following code for an example:

In [ ]:
owm = OWM(OWM_key)
mgr = owm.weather_manager()

In [ ]:
mgr_geo = owm.geocoding_manager()

# geocode Berlin (no country specified) - we'll get many results
list_of_locations = mgr_geo.geocode(cities['City'][0])
city = list_of_locations[0]  # taking the first Berlin in the list
lat = city.lat
lon = city.lon 

In [ ]:
one_call = mgr.one_call(lat=lat, lon=lon, units='metric', timeformat='date')

hourly_forecast = one_call.forecast_hourly

In [ ]:
hourly_forecast #it is a list of Weather objects

In [ ]:
weather_dict = {'city_id': [],
                'forecast_time': [],
                'outlook': [],
                'temperature': [],
                'temperature_feels_like': [],
                'wind_speed': [],
                'pop': []}
for i in range(round(len(hourly_forecast)/3)):
  index = i*3 #because we want forecast for every 3 hours
  weather_dict['city_id'].append(cities['City_id'][0])
  weather_dict['temperature'].append(hourly_forecast[index].temperature().get('temp'))
  weather_dict['wind_speed'].append(hourly_forecast[index].wind().get('speed'))
  weather_dict['forecast_time'].append(hourly_forecast[index].reference_time(timeformat='iso'))
  weather_dict['outlook'].append(hourly_forecast[index].detailed_status)
  weather_dict['temperature_feels_like'].append(hourly_forecast[index].temperature().get('feels_like'))
  weather_dict['pop'].append(hourly_forecast[index].precipitation_probability)



In [ ]:
weather_df = pd.DataFrame(weather_dict)

,city_id,forecast_time,outlook,temperature,temperature_feels_like,wind_speed,pop
0,1,2022-09-30 13:00:00+00:00,few clouds,17.28,16.40,3.04,0.00
1,1,2022-09-30 16:00:00+00:00,few clouds,16.71,15.74,2.59,0.00
2,1,2022-09-30 19:00:00+00:00,clear sky,12.64,11.40,2.39,0.00
3,1,2022-09-30 22:00:00+00:00,clear sky,11.02,9.72,2.86,0.00
4,1,2022-10-01 01:00:00+00:00,few clouds,10.28,9.03,3.22,0.00
5,1,2022-10-01 04:00:00+00:00,broken clouds,10.24,9.07,4.32,0.00
6,1,2022-10-01 07:00:00+00:00,overcast clouds,11.72,10.62,5.02,0.00
7,1,2022-10-01 10:00:00+00:00,light rain,11.08,10.33,6.17,0.73
8,1,2022-10-01 13:00:00+00:00,moderate rain,10.47,9.95,5.52,1.00
9,1,2022-10-01 16:00:00+00:00,light rain,11.50,11.03,5.29,1.00


## 3.2 Approach without a wrapper

OpenWeatherMap API has a nice documentation and you can find it here: https://openweathermap.org/api

We are going to use 5 day / 3 hour forecast https://openweathermap.org/forecast5

Provided at their docmentation you can find an example of API call with only having info about city name and country_code
https://openweathermap.org/forecast5#name5

`api.openweathermap.org/data/2.5/forecast?q={city name},{country code}&appid={API key}`

- city name and country code have to be in format e.g. ('Berin,DE')
- appid is your unique API key
- you can also add units which can be 'standard', 'metric', 'imperial'
- or lang for languages e.g. 'de', 'it' etc.
- or cnt for number of timestamps in response

You can also find an example of API call with having info about latitude and longitude  
https://openweathermap.org/forecast5#geo5

api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API key}

You can choose your own approach. We will show here the approach with city name and country code

#### 3.2.1 Creating a request

In [ ]:
import requests

In [ ]:
url = f"http://api.openweathermap.org/data/2.5/forecast?q={cities['City'][0]},{cities['Country_code'][0]}&appid={OWM_key}&units=metric"
response = requests.get(url)

In [ ]:
response = requests.get(url)
response.status_code

200

#### 3.2.2 Exploring the response (json)

You can copy paste your json response output to one of many online json viewers to see your json in nicer and more practical way
http://jsonviewer.stack.hu/

In [ ]:
response.json()

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1664550000,
   'main': {'temp': 17.26,
    'feels_like': 16.45,
    'temp_min': 16.95,
    'temp_max': 17.26,
    'pressure': 1011,
    'sea_level': 1011,
    'grnd_level': 1006,
    'humidity': 54,
    'temp_kf': 0.31},
   'weather': [{'id': 801,
     'main': 'Clouds',
     'description': 'few clouds',
     'icon': '02d'}],
   'clouds': {'all': 20},
   'wind': {'speed': 3.08, 'deg': 214, 'gust': 4.21},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'd'},
   'dt_txt': '2022-09-30 15:00:00'},
  {'dt': 1664560800,
   'main': {'temp': 15.95,
    'feels_like': 14.98,
    'temp_min': 13.34,
    'temp_max': 15.95,
    'pressure': 1011,
    'sea_level': 1011,
    'grnd_level': 1006,
    'humidity': 53,
    'temp_kf': 2.61},
   'weather': [{'id': 801,
     'main': 'Clouds',
     'description': 'few clouds',
     'icon': '02n'}],
   'clouds': {'all': 18},
   'wind': {'speed': 2.45, 'deg': 183, 'gust': 4.98},
   'visibility': 100

In [ ]:
#This will only work in jupyter notebook so if you are using colab you can skip it

from IPython.display import JSON
# JSON() helps us preview the json in scalable way
JSON(response.json())
# under the node 'list' are the actual weather informations for each hour
JSON(response.json()['list'])

<IPython.core.display.JSON object>

#### 3.2.3 Gathering the data we need from json

In [ ]:
weather_dict = {'city_id': [],
                'forecast_time': [],
                'outlook': [],
                'temperature': [],
                'temperature_feels_like': [],
                'wind_speed': [],
                'pop': []}
# iterating over items in 'list' node and selecting the subnodes gives us the exact info we need 
for hour in response.json()['list']:
  weather_dict['city_id'].append(cities['City_id'][0])
  weather_dict['temperature'].append(hour['main']['temp'])
  weather_dict['wind_speed'].append(hour['wind']['speed'])
  weather_dict['forecast_time'].append(hour['dt_txt'])
  weather_dict['outlook'].append(hour['weather'][0]['description'])
  weather_dict['temperature_feels_like'].append(hour['main']['feels_like'])
  weather_dict['pop'].append(hour['pop'])


#### 3.2.4 Creating a new weather DataFrame

In [ ]:
weather_df = pd.DataFrame(weather_dict)
weather_df

,city_id,forecast_time,outlook,temperature,temperature_feels_like,wind_speed,pop
0,1,2022-09-30 15:00:00,few clouds,17.26,16.45,3.08,0.00
1,1,2022-09-30 18:00:00,few clouds,15.95,14.98,2.45,0.00
2,1,2022-09-30 21:00:00,few clouds,13.39,12.27,2.62,0.00
3,1,2022-10-01 00:00:00,clear sky,10.49,9.19,2.80,0.00
4,1,2022-10-01 03:00:00,scattered clouds,10.04,8.88,4.19,0.00
5,1,2022-10-01 06:00:00,broken clouds,10.83,9.69,4.97,0.00
6,1,2022-10-01 09:00:00,overcast clouds,13.81,12.76,6.77,0.28
7,1,2022-10-01 12:00:00,moderate rain,10.19,9.64,5.86,1.00
8,1,2022-10-01 15:00:00,moderate rain,10.99,10.55,5.13,1.00
9,1,2022-10-01 18:00:00,light rain,11.87,11.41,5.22,1.00
